The classical baseline is computed in the cell below.

In [1]:
import numpy as np

from inventory_env_hard.env_creator import inventory_env_hard_creator
from inventory_env_hard.utils import classical_baseline_action

env = inventory_env_hard_creator(config={}, seed=0)
num_episodes = 100000
all_r = []
for _ in range(num_episodes):
    obs = env.reset()
    ep_r = 0
    while True:
        action = classical_baseline_action(obs)
        obs, r, done, _ = env.step(np.around(action))
        ep_r += r
        if done:
            break
    all_r.append(ep_r)
baseline = sum(all_r) / num_episodes
print(baseline)

/home/dibya/miniconda3/envs/real_world_deep_rl_course_cpu/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


168462.89666968363


Here's how you restore your best agent

In [ ]:
from pathlib import Path

from ray.rllib.agents.ppo import PPOTrainer
from ray.tune import ExperimentAnalysis
from ray.tune.registry import register_env

from inventory_env_hard.env_creator import inventory_env_hard_creator

register_env("inventory_env_hard", inventory_env_hard_creator)

# Here you need to use the path to the Ray Tune results dir on YOUR COMPUTER
path_to_results_dir = Path().absolute().parents[1] / "exercises" / "solutions" / "experiment_results" / "experiment_many_samples"
analysis = ExperimentAnalysis(path_to_results_dir, default_metric="evaluation/episode_reward_mean", default_mode="max")
best_trial = analysis.get_best_trial(scope="all")
best_checkpoint = analysis.get_best_checkpoint(best_trial)
config = best_trial.config
agent = PPOTrainer(config=config)
agent.restore(best_checkpoint)

Then compute the agent's performance and compare with the baseline.

In [ ]:
# Remember to use the exact same observation wrapper that the restored agent used during training
# This may be different in your case
env = inventory_env_hard_creator(
    {
        "obs_filter": "my_normalize",    
        "reward_filter": None, 
    }, 
    seed=0
)
num_episodes = 100000
all_r = []
for _ in range(num_episodes):
    obs = env.reset()
    ep_r = 0
    while True:
        action = agent.compute_action(obs, unsquash_action=True)
        obs, r, done, _ = env.step(np.around(action))
        ep_r += r
        if done:
            break
    all_r.append(ep_r)
baseline = sum(all_r) / num_episodes
print(baseline)